In [2]:
# Библиотеки

from scipy.integrate import solve_ivp
import numpy as np
import matplotlib.pyplot as plt

import torch 
import torch.optim as optim
# from torchdiffeq import odeint             # пока не понятно для чего

In [3]:
# Чтобы при вызове print или display после точки выводилось фиксированное число знаков

np.set_printoptions(precision=5, suppress=True)

In [4]:
# Переменные

IS_RESTART = False
P_CUTOFF = 0.0                                                     # Непонятно для чего
N_EPOCH = 1000                                                     # Количество эпох. Максимальное
N_PLOT = 100                                                       # Частота формирования графиков. Через сколько эпох

# Есть большая вероятность, что нам не нужен будет OPT, и что строчку ниже нужно будет удалить
OPT = optim.AdamW([torch.tensor(1.0)], lr=0.001, betas=(0.9, 0.999), eps=1e-8)          # Оптимизатор
DATASIZE = 10                                                      # Размер датасетов?
TSTEP = 1                                                          # Шаг времени для датасетов? или типа их количество?
N_EXP_TRAIN = 6                                                    # Размер данных для обучения
N_EXP_TEST = 2                                                     # Размер даных для теста
N_EXP = N_EXP_TRAIN + N_EXP_TEST                                   # Общий размер данных
NOISE = 5e-2                                                       # ШУМ
NS = 5                                                             # Количество веществ
NR = 4                                                             # Количество хим. реакций
K = torch.tensor([0.1, 0.2, 0.13, 0.3], dtype=torch.float32)                                   # константы хим. реакций
ATOL = 0.00001                                                     # Параметр точности для ОДУ
RTOL = 0.01                                                        # Параметр точности для ОДУ
B0 = -10.0 

MAXITERS = 10000                                                   # Не понял для чего нужно

LB = 0.00001
UB = 1


In [5]:
u0_list = np.array([
    [0.2, 0.4, 0.0, 0.0, 0.0],
    [1.0, 0.7, 0.0, 0.0, 0.0],
    [0.8, 0.2, 0.0, 0.0, 0.0],
    [0.4, 0.2, 0.0, 0.0, 0.0],
    [0.4, 0.7, 0.0, 0.0, 0.0],
    [0.4, 0.6, 0.0, 0.0, 0.0],
    [0.7, 0.2, 0.0, 0.0, 0.0],
    [0.6, 1.1, 0.0, 0.0, 0.0]
])

# u0_list = np.random.rand(N_EXP, NS).astype(np.float32)
u0_list[:, 0:2] += 0.2
u0_list[:, 2:] = 0.0

tspan = [0.0, DATASIZE * TSTEP]
tsteps = np.linspace(tspan[0], tspan[1], DATASIZE)               # возможно надо оставить просто DATASIZE
ode_data_list = np.zeros((DATASIZE, N_EXP, NS), dtype=np.float32)


std_list = []

In [6]:
print(ode_data_list)

[[[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]


In [7]:
def true_ode_func(t, y, k):
    """
    Вычисляет производные системы дифференциальных уравнений.
    
    Параметры:
    t : float
        Время (не используется в данном уравнении, но необходим для совместимости с solve_ivp).
    y : array-like
        Вектор переменных (y[0], y[1], ..., y[4]).
    k : array-like
        Коэффициенты (k[0], k[1], ..., k[4]).

    Возвращает:
    dydt : numpy.ndarray
        Вектор производных.
    """
    dydt = np.zeros_like(y)
    dydt[0] = -2 * k[0] * y[0]**2 - k[1] * y[0]
    dydt[1] = k[0] * y[0]**2 - k[3] * y[1] * y[3]
    dydt[2] = k[1] * y[0] - k[2] * y[2]
    dydt[3] = k[2] * y[2] - k[3] * y[1] * y[3]
    dydt[4] = k[3] * y[1] * y[3]
    return dydt


In [ ]:
std_list = []

for i in range(N_EXP):
    u0 = u0_list[i, :]  # Начальные условия
    # Решение
    sol = solve_ivp(
        true_ode_func,              # Функция ОДУ
        tspan,                      # Интервал времени
        u0,                         # Начальные условия
        args=(K,),                  # Передача параметров в функцию
        method='RK45',              # Метод решения
        t_eval=tsteps               # Точки, в которых сохраняется решение (Шаг)
    )
    # Преобразование решения
    ode_data = sol.y.T  # Транспонируем для согласования с форматом (в Julia это Array)
    

    ode_data_list[:, i, :] = ode_data
    # std_list.append(np.max(ode_data, axis=1) - np.min(ode_data, axis=1) + LB)      # С добавлением шума
    std_list.append(np.max(ode_data, axis=0) - np.min(ode_data, axis=0))         # Без добавления шума

# Вычисляем y_std
std_matrix = np.column_stack(std_list)  # Аналог hcat в Julia
y_std = np.max(std_matrix, axis=1)

In [9]:
print(u0_list[0,:])

[0.4 0.6 0.  0.  0. ]


In [17]:
print(ode_data_list[:,0,:])
print(ode_data_list[:,1,:])

[[0.4     0.6     0.      0.      0.     ]
 [0.29665 0.61296 0.07115 0.00517 0.00037]
 [0.22428 0.61825 0.11488 0.01669 0.00255]
 [0.17191 0.61773 0.14001 0.03044 0.0074 ]
 [0.1331  0.61264 0.15244 0.04403 0.01505]
 [0.10382 0.60399 0.15625 0.05621 0.02525]
 [0.08145 0.59268 0.15423 0.06643 0.03751]
 [0.06418 0.57946 0.14845 0.07452 0.05131]
 [0.05074 0.565   0.14028 0.08057 0.06614]
 [0.04023 0.54985 0.13074 0.08478 0.08152]]
[[1.2     0.9     0.      0.      0.     ]
 [0.77546 1.0034  0.19859 0.01414 0.00167]
 [0.53784 1.04091 0.30508 0.04188 0.01112]
 [0.38898 1.04477 0.35849 0.07062 0.0308 ]
 [0.28908 1.02854 0.37953 0.09537 0.05965]
 [0.21888 0.99995 0.38046 0.11476 0.09533]
 [0.16795 0.96404 0.36892 0.12897 0.13536]
 [0.13013 0.92425 0.34988 0.1387  0.1776 ]
 [0.10161 0.88296 0.32655 0.14486 0.22035]
 [0.07975 0.84172 0.30124 0.14806 0.26251]]


In [16]:
print(ode_data_list)

[[[0.4     0.6     0.      0.      0.     ]
  [1.2     0.9     0.      0.      0.     ]
  [1.      0.4     0.      0.      0.     ]
  [0.6     0.4     0.      0.      0.     ]
  [0.6     0.9     0.      0.      0.     ]
  [0.6     0.8     0.      0.      0.     ]
  [0.9     0.4     0.      0.      0.     ]
  [0.8     1.3     0.      0.      0.     ]]

 [[0.29665 0.61296 0.07115 0.00517 0.00037]
  [0.77546 1.0034  0.19859 0.01414 0.00167]
  [0.66768 0.47462 0.16838 0.01265 0.00069]
  [0.42914 0.42858 0.10473 0.00781 0.00039]
  [0.42914 0.92815 0.10473 0.00739 0.00081]
  [0.42914 0.82824 0.10473 0.00747 0.00073]
  [0.61107 0.46138 0.15288 0.01147 0.00061]
  [0.55251 1.34829 0.13711 0.0093  0.0015 ]]

 [[0.22428 0.61825 0.11488 0.01669 0.00255]
  [0.53784 1.04091 0.30508 0.04188 0.01112]
  [0.47186 0.50583 0.26156 0.04014 0.00493]
  [0.31656 0.44149 0.16677 0.02548 0.00274]
  [0.31656 0.93882 0.16677 0.0228  0.00541]
  [0.31656 0.83932 0.16677 0.0233  0.00491]
  [0.4362  0.48763 0.23888 0

## <span style="color: red;">Важная заметка!!!</span>
sol = solve_ivp(true_ode_func, tspan, u0, args=(K,), method='RK45', t_eval=tsteps) <br>
ode_data = sol.y.T <br>

ode_data - матрица решений ОДУ
## <span style="color: red;">Важная заметка!!!</span>

## Всё что выше, работает!

In [80]:
def p2vec(p):
    """
    Конвертирует параметры `p` в смещения, веса и ограниченные значения весов.

    Args:
        p (array-like): Вектор параметров.
        NR (int): Количество элементов в `w_b`.
        NS (int): Количество строк в `w_out`.
        B0 (float): Сдвиг для вектора смещений.

    Returns:
        tuple: Три массива: w_in, w_b, w_out.
    """
    # Вектор смещений
    # w_b = p[:NR] + B0
    w_b = p[:NR]

    # Матрица будущих весов
    w_out = np.reshape(p[NR:], (NS, NR))

    # Ограничение значений весов
    w_in = np.where(w_out > 0, 0, -w_out)

    return w_in, w_b, w_out

def display_weight(p):
    """
    Выводит параметры w_in, w_b и w_out в читабельном формате.

    Args:
        p (array-like): Вектор параметров (одномерный массив).
    """
    # Извлечение параметров с помощью p2vec
    w_in, w_b, w_out = p2vec(p)

    print('w_in', w_in, end='\n\n')
    print('w_b', w_b, end='\n\n')
    print('w_out', w_out, end='\n\n')

    print("Species (column) reaction (row)\n")
    
    # Вывод w_in
    print("w_in")
    print(np.round(w_in, 5))  # Транспонирование для совпадения с Julia
    
    # Вывод w_b
    print("\nw_b")
    print(np.round(np.exp(w_b), 5))  # Применяем экспоненту и округление

    # Вывод w_out
    print("\nw_out")
    print(np.round(w_out, 5))  # Транспонирование для совпадения с Julia

    print("\n")




In [75]:
# Функция для обновления du
def crnn(du, u, p, t):
    """
    Вычисляет производную du для системы ОДУ.

    Args:
        du (array-like): Выходной массив для производной.
        u (array-like): Текущее значение u.
        p (dict): Параметры модели.
        t (float): Текущее время.
    """
    # Вычисляем параметры через p2vec
    w_in, w_b, w_out = p2vec(p)

    # Логарифм ограниченных значений
    u_clamped = np.clip(u, LB, UB)
    w_in_x = np.dot(w_in.T, np.log(u_clamped))

    # Экспонента и обновление du
    du[:] = np.dot(w_out, np.exp(w_in_x + w_b))


# Обертка для solve_ivp
def crnn_wrapper(t, u, p):
    """
    Обертка для функции crnn для использования в solve_ivp.

    Args:
        t (float): Текущее время.
        u (array-like): Текущее значение u.
        p (dict): Параметры модели.

    Returns:
        array-like: Производная u.
    """
    du = np.zeros_like(u)
    crnn(du, u, p, t)
    return du

In [83]:
p = [1.0,2.0,3.0,4.0,5.0,6.0,7.0,6.0,5.0,4.0,3.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,4.0,2.0,4.0,9.0,4.0]
u0 = u0_list[0, :]
print(u0)

[0.4 0.6 0.  0.  0. ]


In [81]:

def crnn(t, u, p):
    w_in, w_b, w_out = p2vec(p)
    u_clamped = np.clip(u, LB, UB)
    w_in_x = np.dot(w_in.T, np.log(u_clamped))
    du = np.dot(w_out, np.exp(w_in_x + w_b))
    return du

# Решение задачи
sol = solve_ivp(crnn, tspan, u0, args=(p,), t_eval=tsteps, atol=1e-6, rtol=1e-3)


<br><br><br><br>
Функция `predict_neuralode` из Julia у нас заложена в `solve_ivp`
<br><br><br><br><br>

In [82]:
print(sol.y)

[[   0.4      584.97045 1169.5409  1754.11135 2338.6818  2923.25225
  3507.8227  4092.39315 4676.96359 5261.53404]
 [   0.6      236.82283  473.04565  709.26848  945.49131 1181.71414
  1417.93696 1654.15979 1890.38262 2126.60544]
 [   0.       517.47517 1034.95034 1552.42552 2069.90069 2587.37586
  3104.85103 3622.3262  4139.80138 4657.27655]
 [   0.       530.3365  1060.67301 1591.00951 2121.34602 2651.68252
  3182.01903 3712.35553 4242.69204 4773.02854]
 [   0.       482.39469  964.78938 1447.18407 1929.57876 2411.97345
  2894.36814 3376.76283 3859.15752 4341.5522 ]]
